In [1]:
import torch
import numpy as np
from scipy.stats import multivariate_normal

In [2]:
from gibbsibp import UncollapsedGibbsIBP

In [98]:
F_np = np.array(
    [[0.51, 0.0],
     [0.49, 0.0]]
    )
X_np = np.array(
    [[0.0, 0.0],
     [0.0, 0.0]]
    )

A_np = np.array([[0.4, 0.0]])
Z_np = np.array(
    [[1.0], 
     [1.0]]
    )

SIGMA2_N = 0.05
SIGMA2_A = 1.0


F_torch = torch.tensor(F_np, dtype=torch.float32)
X_torch = torch.tensor(X_np, dtype=torch.float32)
A_torch = torch.tensor(A_np, dtype=torch.float32)
Z_torch = torch.tensor(Z_np, dtype=torch.float32)

inf = UncollapsedGibbsIBP(K=1, max_K=6, alpha=0.01, sigma2_a=1.0, phi=0.25, sigma2_n=SIGMA2_N, epsilon=0.25, lambd=0.75)

As, Zs, Ys = inf.gibbs(F_torch, X_torch, 0)

In [58]:
# Calculate log likelihood using the implementation in the class

for i in range(F_torch.shape[0]):

    my_loglik = inf.loglik_f_i__given_Az(F_torch[i:i+1,:], A_torch, Z_torch[i:i+1,:]).numpy()[0]

    # Compare results with the log likelihood calculated in scipy
    mean = (Z_torch[i:i+1]@A_torch).numpy()[0]
    cov = np.eye(2)*SIGMA2_N
    multivariate_normal.logpdf(A_np, mean=mean, cov=cov)

    sp_loglik = multivariate_normal.logpdf(F_np[i,:], mean, cov)

    if np.isclose(my_loglik, sp_loglik, atol=1e-6):
        print(f"Test passed for sample {i}")

Test passed for sample 0
Test passed for sample 1


In [103]:
inf.resample_A(F_torch, Z_torch)
temp = (Z_torch.T @ Z_torch + (SIGMA2_N/SIGMA2_A)*torch.eye(2)).inverse()
mu = temp @ Z_torch.T @ F_torch

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 1x2)

np.float64(1.1578552071446455)